In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import sympy as sym
import time 

## Newton's Method
Solve system of nonlinear equations. 
$$
    3x_1 - \cos (x_2 x_3) - \frac{1}{2} = 0\\
    x_1^2 - 81(x_2 + 0.1)^2 + \sin(x_3) + 1.06 = 0\\
    e^{-x_1 x_2} + 20x_3 + \frac{10\pi - 3}{3} = 0
$$

In [3]:
x1, x2, x3 = sym.symbols('x_1, x_2, x_3')
ind_vars = np.array([x1, x2, x3])

In [4]:
eq1 = 3*x1 - sym.cos(x2*x3) - 1/2
eq2 = x1**2 - 81*(x2 + 0.1)**2 + sym.sin(x3) + 1.06 
eq3 = sym.exp(-x1*x2) +20*x3 + 1/3*(10*np.pi - 3)

sys_eqn = np.array([eq1, eq2, eq3])

In [5]:
type(eq1)

sympy.core.add.Add

### Step 1: Propose an initial guess

In [6]:
class solution:
    def __init__(self, solution):
        self.solution = solution
        
        self.counter = 0
        self.error = 1.0E6
        self.run_time = 0.0 

In [7]:
x_guess = np.array([0.1, 0.1, -0.1])

### Step 2: Define F(x) and J(x)

In [8]:
def F_x_num(sys_eqn, ind_vars, vector):
    num_array = np.copy(sys_eqn)
    num_array_final = np.zeros(len(sys_eqn), dtype = 'float')
    
    for i in range(len(sys_eqn)):
        for j in range(len(ind_vars)):
            num_array[i] = num_array[i].subs(ind_vars[j], vector[j])
        num_array_final[i] = sym.N(num_array[i])
    return num_array_final

In [9]:
def J_x(sys_eqn, ind_vars):
    Jacobian = np.zeros((len(sys_eqn), len(ind_vars)), dtype='object')
    
    for i in range(len(sys_eqn)):
        for j in range(len(ind_vars)):
            Jacobian[i,j] = sym.diff(sys_eqn[i], ind_vars[j])
    
    return Jacobian

In [10]:
def J_x_num(sys_eqn, ind_vars, vector):
    # substitute in numerical values
    
    A = J_x(sys_eqn, ind_vars)
    
    for i in range(len(A[0][:])):
        for j in range(len(A[:][0])):
            for k in range(len(ind_vars)):
                A[i][j] = A[i][j].subs(ind_vars[k], vector[k])
                
    return np.array(A, dtype = 'float')

### Step 3: Solve System via Gaussian Elimination

In [11]:
Jx = J_x_num(sys_eqn, ind_vars, x_guess)
Fx_min = -F_x_num(sys_eqn, ind_vars, x_guess)
np.linalg.solve(Jx, Fx_min)

array([ 0.39986967, -0.08053315, -0.42152047])

### Step 4: Compute/Update Guess

In [12]:
x_guess = x_guess + np.linalg.solve(Jx, Fx_min)

In [13]:
x_initial = x_guess

In [14]:
x_guess

array([ 0.49986967,  0.01946685, -0.52152047])

### Step 5: Continue
1. Build in a condition that if the change becomes too small, then we're done. This is given by the condition $||x^{(k)} - x^{(k-1)}|| < \epsilon$ for some acceptable error $\epsilon$. Since $x$ is a one dimensional vector, we can take this to be the dot product norm.

In [15]:
def err(x_1, x_2):
    diff = x_1 - x_2
    return np.sqrt(np.dot(diff,diff))

In [16]:
err(x_guess, x_initial)

0.0

In [17]:
def Newton(sys_eqn, ind_vars, initial_guess, accept_error = 1.0E-6):
    
    # timer variables 
    start = time.time()
    end = 0.0 
    run_time = 60.0 # seconds 
    
    # declare solution object 
    answer = solution(initial_guess)
    
    while(answer.error > accept_error):
        Jacobian = J_x_num(sys_eqn, ind_vars, answer.solution)
        Fx_min = -F_x_num(sys_eqn, ind_vars, answer.solution)
        y = np.linalg.solve(Jx, Fx_min)
        
        prior_solution = answer.solution
        answer.solution = answer.solution + y
        answer.counter += 1 
        answer.error = err(answer.solution, prior_solution) 
        
        print(answer.error)
        
        end = time.time()
        answer.run_time = end - start 
        
        if (end - start > run_time):
            print("Not converged")
            break 
            
        
    
    return answer

In [18]:
test = Newton(sys_eqn, ind_vars, x_guess)

0.010805894452734425
0.004589031712055804
0.0021469533972104256
0.0010408017920779853
0.0005128900988624824
0.0002547434651124233
0.0001270171858396936
6.345345521283225e-05
3.172950027048003e-05
1.5873712312334434e-05
7.943235223639728e-06
3.9752842549083765e-06
1.9895959873673657e-06
9.958056526433444e-07


In [19]:
test.solution

array([ 5.00000000e-01,  9.96855777e-07, -5.23598731e-01])

## Broyden's Method
$$
    x^{(i+1)} = x^{(i)} - A_i^{-1} F(x^{(i)})
$$
with
$$
    A_i^{-1} = \left(A_{i-1} + \frac{y_i - A_{i-1}s_i}{||s_i||^2_2}s_i^t\right)\\
    = A_{i-1}^{-1} + \frac{(s_i - A_{i-1}^{-1}y_i)s_i^t A_{i-1}^{-1}}{s_i^t A_{i-1}^{-1}y_i}
$$

#### Step 1: Initial Guess: $x^{(0)}$

In [20]:
x_guess = np.array([0.1, 0.1, -0.1])

#### Step 2: Calculate $F(x^{(0)})$

In [31]:
sys_eqn = np.array([eq1, eq2, eq3])
F_x_0 = F_x_num(sys_eqn, ind_vars, x_guess)

#### Step 3: Calculate $A_0^{-1}$
1. At the first step, we take $A_0 = J(x^0)$

In [30]:
A_0 = J_x_num(sys_eqn, ind_vars, x_guess)
A_0_inv = np.linalg.pinv(A_0)

#### Step 4: Calculate: $x^1 = x^0 - A_0^{-1}F(x^0)$

In [36]:
x_new = x_guess - np.matmul(A_0_inv, F_x_0)
x_new

array([ 0.49986967,  0.01946685, -0.52152047])

#### Step 5: Calculate $F(x^1)$

In [37]:
F_x_1 = F_x_num(sys_eqn, ind_vars, x_new)

#### Step 6: Calculate $y_1 = F(x^1) - F(x^0)$ and $s_1 = x^1 - x^0$

In [39]:
y1 = F_x_1 - F_x_0
y1

array([ 1.19961055,  1.92544549, -8.43014297])

In [41]:
s1 = x_new - x_guess
s1

array([ 0.39986967, -0.08053315, -0.42152047])

#### Step 7: Calculate $s_1^t A_0^{-1}y_1$

In [43]:
prod = np.matmul(s1,np.matmul(A_0_inv, y1))
prod

0.3424603869557373

### Step 8: Find
$$
    A_1^{-1} = A_0^{-1} + \left(s_1^t A_0^{-1}y_1\right)^{-1} 
    \left[(s_1 - A_0^{-1} y_1)s_1^t A_0^{-1}\right]
$$

In [65]:
One = s1 - np.matmul(A_0_inv, y1)
Two = np.matmul(s1, A_0_inv)
Three = np.outer(One, Two)
A_1_inv = A_0_inv + 1/prod*Three
A_1_inv

array([[ 3.33378100e-01,  1.11049659e-05,  8.96734389e-06],
       [-2.02070982e-03, -3.09484821e-02,  2.19681582e-03],
       [ 1.02389942e-03, -1.65038427e-04,  5.01095867e-02]])

#### Step 9: Find 
$$
    x^2 = x^1 - A_1^{-1}F(x^1)
$$

In [69]:
x_new = x_new - np.matmul(A_1_inv, F_x_1)
x_new

array([ 0.50010308, -0.00199117, -0.52482868])

In [ ]:
def broyden(sys_eqn, ind_vars, initial_guess, accept_error = 1.0E-6):
    # timer variables 
    start = time.time()
    end = 0.0 
    run_time = 60.0 # seconds 
    
    # declare solution object 
    answer = solution(initial_guess)
    
    # 